# Logistic Regression
This notebook explores logistic regression to predict shipment delays using a supply chain logistics dataset.

Objectives:
Preprocess the dataset for machine learning.
Implement logistic regression using sklearn and a custom implementation.
Evaluate the models' performance.
Compare different approaches for feature selection and model evaluation.

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle

## 1. Data loading
We will load the dataset and perform a preliminary check for missing values or inconsistencies.

In [4]:
# Load the dataset
file_path = '/Users/ceciliaalberti/Documents/INDE_577/datasets/Supply chain logisitcs problem.xlsx'
df = pd.read_excel(file_path)
df.head()

,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.30
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.94
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.20
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.16
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.34


## 2. Data preprocessing
This section handles:

1. Creating the target variable.
2. Selecting features and target.
3. Scaling numerical features
4. Encoding categorical features.

In [12]:
df.info

<bound method DataFrame.info of           Order ID Order Date Origin Port Carrier  TPT Service Level  \
0     1.447296e+09 2013-05-26      PORT09   V44_3    1           CRF   
1     1.447158e+09 2013-05-26      PORT09   V44_3    1           CRF   
2     1.447139e+09 2013-05-26      PORT09   V44_3    1           CRF   
3     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
4     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
...            ...        ...         ...     ...  ...           ...   
9210  1.447305e+09 2013-05-26      PORT04  V444_1    1           DTD   
9211  1.447319e+09 2013-05-26      PORT04  V444_1    1           DTD   
9212  1.447322e+09 2013-05-26      PORT04  V444_1    1           DTD   
9213  1.447145e+09 2013-05-26      PORT04  V444_1    1           DTD   
9214  1.447328e+09 2013-05-26      PORT04  V444_1    1           DTD   

   Ship ahead day count  Ship Late Day count   Customer  Product ID  \
0                     3         

In [9]:
# Create the target column: 1 for delayed shipments, 0 for on-time shipments
df['Delayed'] = df['Ship Late Day count'].apply(lambda x: 1 if x > 0 else 0)

# Select features and target
features = ['Origin Port', 'Carrier', 'Ship ahead day count', 'Origin Port', 'Plant Code', 'Destination Port', 'Weight']
X = df[features]
y = df['Delayed']


In [10]:
def standard_scaler(X, cols_to_scale):
    X_scaled = X.copy()
    for col in cols_to_scale:
        mean = X[col].mean()  # Calculate mean
        std = X[col].std()    # Calculate standard deviation
        X_scaled[col] = (X[col] - mean) / std  
    return X_scaled

In [11]:
# Identify numerical features to scale
numerical_features = ['Ship ahead day count', 'Weight']

# Apply manual scaling to numerical features
X_scaled = standard_scaler(X, numerical_features)

# Encode categorical features (leave scaled numerical features unchanged)
X_encoded = pd.get_dummies(X_scaled, columns=['Origin Port', 'Carrier', 'Plant Code', 'Destination Port'], drop_first=True)

## 2. Custom Logistic Regression Implementation
We implement logistic regression from scratch using gradient descent for better understanding.

In [13]:
class GradientDescentLogisticReg:
    def __init__(self) -> None:
        self.X = None
        self.variables = None
        self.y = None
        self.predictor = None
        self.n = None
        self.p = None
        self.bias = None
        self.gamma = None
        self.max_iter = None
        self.eta = None
        self.weights = None
        self.weights_history = []
        self.loss_history = [np.inf]

    # Cross entropy loss for one data point
    def cross_entropy_loss(self, y, y_hat):
        epsilon = 1e-10  # Small value to avoid log(0)
        y_hat = np.clip(y_hat, epsilon, 1 - epsilon)  # Clip y_hat to avoid extreme values
        return -y * np.log(y_hat) - (1.0 - y) * np.log(1.0 - y_hat)

    # Total cross entropy loss
    def loss(self):
        total_loss = sum(self.cross_entropy_loss(self.y[i], self.sigmoid(x @ self.weights))
                         for i, x in enumerate(self.X))
        return total_loss

    # Sigmoid function
    def sigmoid(self, z):
        # Clip the input values to avoid extremely large values that cause overflow
        z = np.clip(z, -500, 500)
        return 1.0 / (1.0 + np.exp(-z))

    # Gradient of the loss function
    def gradient_L(self):
        sigmoids = np.array([self.sigmoid(x @ self.weights) - self.y[i] for i, x in enumerate(self.X)])
        d_w = sigmoids @ self.X
        return d_w

    # Model fitting with gradient descent
    def fit(self, X, y, bias=True, gamma=0.01, max_iter=1000, eta=0.001, patience=10):
        self.variables = X.columns
        self.predictor = y.name

        X = X.to_numpy()
        y = y.to_numpy()
        if bias:
            ones_column = np.ones((X.shape[0], 1))
            X = np.append(ones_column, X, axis=1)
        self.X = X
        self.y = y
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.bias = bias
        self.gamma = gamma
        self.max_iter = max_iter
        self.eta = eta

        weights = np.random.rand(self.p) * 0.01  # Initialize weights small but non-zero
        self.weights = weights
        self.weights_history.append(weights)

        patience_counter = 0

        for i in range(1, max_iter + 1):
            dw = self.gradient_L()
            weights = weights - gamma * dw
            self.weights = weights
            self.weights_history.append(weights)
            L = self.loss()
            self.loss_history.append(L)

            # Print weights every 100 iterations for debugging
            if i % 100 == 0:
                print(f"Iteration {i}: Loss = {L}, Weights = {self.weights[:5]}...")

            # Adaptive learning rate (reduce gamma if loss does not decrease sufficiently)
            if i > 1 and L > self.loss_history[i - 1]:
                gamma *= 0.9  # Reduce learning rate by 10% if loss increases

            # Early stopping with patience
            if i > 1 and abs(L - self.loss_history[i - 1]) <= self.eta:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Stopping early at iteration {i}")
                    break
            else:
                patience_counter = 0

    # Predict new data
    def prediction(self, X, weights=None):
        if weights is None:
            weights = self.weights

        X = X.to_numpy()
        if self.bias:
            ones_column = np.ones((X.shape[0], 1))
            X = np.append(ones_column, X, axis=1)

        labels = np.array([1, 0])
        y_hat = [self.sigmoid(x @ weights) for x in X]
        return [np.random.choice(labels, p=[y_hat_i, 1.0 - y_hat_i]) for y_hat_i in y_hat]
    
    # Calculate accuracy
    def accuracy(self, y_true, y_pred):
        correct_predictions = sum(y_true == y_pred)  # Count correct predictions
        total_predictions = len(y_true)             # Total number of predictions
        accuracy = correct_predictions / total_predictions
        return accuracy

### Downsampling

In [27]:
# Downsample the majority class (shipments NOT delayed, target 0) to 1000 data points
majority_class = X_encoded[y == 0]
minority_class = X_encoded[y == 1]
y_majority = y[y == 0]
y_minority = y[y == 1]

majority_downsampled = majority_class.sample(n=1000, random_state=42)
y_majority_downsampled = y_majority.sample(n=1000, random_state=42)

X_downsampled = pd.concat([majority_downsampled, minority_class]).reset_index(drop=True)
y_downsampled = pd.concat([y_majority_downsampled, y_minority]).reset_index(drop=True)

# Shuffle the downsampled dataset
X_downsampled, y_downsampled = shuffle(X_downsampled, y_downsampled, random_state=42)

### SMOTE

In [28]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_encoded, y)

### Downsampling + PCA

In [29]:
# Apply PCA for dimensionality reduction
pca = PCA(n_components=0.9)  # Retain 90% of the variance
X_pca_downsampling = pca.fit_transform(X_downsampled)

### SMOTE + PCA

In [30]:
# Apply PCA for dimensionality reduction
pca = PCA(n_components=0.9)  # Retain 90% of the variance
X_pca_smote = pca.fit_transform(X_resampled)

## 4. Model Training and Evaluation
We train the model and evaluate its performance using accuracy, classification report and confusion matrix. We will perform the model under different circumstances:
1. With downsampled data for the majority class
2. With SMOTE
3. With downsampling + PCA
4. With SMOTE + PCA

### Downsampling results

In [31]:
# Split the resampled dataset into training and testing sets
# The dataset is split into training and testing sets to evaluate the model.
X_train, X_test, y_train, y_test = train_test_split(X_downsampled, y_downsampled, test_size=0.2, random_state=42)

# Fit the custom logistic regression model
# The model is fitted using the training data. Gradient descent is used to optimize the model parameters.
gd_log_reg = GradientDescentLogisticReg()
gd_log_reg.fit(pd.DataFrame(X_train), pd.Series(y_train), gamma=0.01, max_iter=1000, eta=0.001)

# Predict the test set
# Predictions are made for the test set using the fitted model.
y_pred = gd_log_reg.prediction(pd.DataFrame(X_test))

# Calculate accuracy
# The accuracy of the model on the test set is calculated.
accuracy = gd_log_reg.accuracy(y_test, y_pred)
print("Accuracy of Gradient Descent Logistic Regression Model:", accuracy)

# Classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Classification report and confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Iteration 100: Loss = 376.8161626612966, Weights = [-1.591396534592121 0.8142054052232357 -0.3710690083416482
 0.0028314030903280043 -0.9332829132874453]...
Iteration 200: Loss = 376.1443931838316, Weights = [-1.586093584127645 0.8197649550383693 -0.6923369394938117
 0.0028314030903280043 -1.2736020109283723]...
Iteration 300: Loss = 375.8412381650964, Weights = [-1.5832448211390822 0.8220700701321326 -0.9974842764099426
 0.0028314030903280043 -1.5142285230564487]...
Iteration 400: Loss = 375.64335901538794, Weights = [-1.5811041630510634 0.8236241720742584 -1.2897340150060108
 0.0028314030903280043 -1.7042377983016141]...
Iteration 500: Loss = 375.4934835626414, Weights = [-1.5792943584041677 0.8248275469739154 -1.5706546980622638
 0.0028314030903280043 -1.8622862510191343]...
Iteration 600: Loss = 375.37103616975594, Weights = [-1.5776809918341166 0.825824876377366 -1.8413574468417957
 0.0028314030903280043 -1.997971678390136]...
Stopping early at iteration 691
Accuracy of Gradient D

The accuracy is high because class 0 (on-time shipments) dominates, and the model performs well for class 0. However, the recall for class 1 (delayed shipments) is low, meaning that the model struggles to detect delayed shipments. The problem is that the model is biased towards predicting on-time shipments, which indicates that there is stil a class imbalance that is causing the model to perform poorly on the minority class.

### SMOTE results

In [32]:
# Split the resampled dataset into training and testing sets
# The dataset is split into training and testing sets to evaluate the model.
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Fit the custom logistic regression model
# The model is fitted using the training data. Gradient descent is used to optimize the model parameters.
gd_log_reg = GradientDescentLogisticReg()
gd_log_reg.fit(pd.DataFrame(X_train), pd.Series(y_train), gamma=0.01, max_iter=1000, eta=0.001)

# Predict the test set
# Predictions are made for the test set using the fitted model.
y_pred = gd_log_reg.prediction(pd.DataFrame(X_test))

# Calculate accuracy
# The accuracy of the model on the test set is calculated.
accuracy = gd_log_reg.accuracy(y_test, y_pred)
print("Accuracy of Gradient Descent Logistic Regression Model:", accuracy)

# Classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Classification report and confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Iteration 100: Loss = 8840.676637539924, Weights = [0.21175171101803553 0.6879588846425251 -2.540405900819591
 -0.04771488775043425 -6.302437188500938]...
Iteration 200: Loss = 8831.910223329374, Weights = [0.17082715981050947 0.6331369855972377 -2.9623580170150725
 -0.04771488775043772 -6.3067048580115745]...
Iteration 300: Loss = 8828.80931897217, Weights = [0.17301536745119658 0.6326493483787606 -3.365185093614477
 -0.04771488775044119 -6.310922561911428]...
Iteration 400: Loss = 8825.975988936469, Weights = [0.17509558232924696 0.632195081436683 -3.750239477794617
 -0.04771488775044466 -6.315099491275461]...
Iteration 500: Loss = 8823.380904495243, Weights = [0.17707512098913353 0.6317711015555056 -4.118748487112882
 -0.04771488775044813 -6.319238625535772]...
Iteration 600: Loss = 8820.998323741345, Weights = [0.17896087959942858 0.6313749435074938 -4.471846190417867
 -0.0477148877504516 -6.323342506120749]...
Iteration 700: Loss = 8818.805714525957, Weights = [0.1807593367144927 

After SMOTE, the dataset is balanced, which theoretically helps with classification. However, it also introduces synthetic examples that might not improve the model’s ability to generalize if the feature relationships are not well captured. This is evident from the balanced but poor performance (precision, recall, F1-score all around 0.57).

### Downsampling + PCA results

In [33]:
# Split the resampled dataset into training and testing sets
# The dataset is split into training and testing sets to evaluate the model.
X_train, X_test, y_train, y_test = train_test_split(X_pca_downsampling, y_downsampled, test_size=0.2, random_state=42)

# Fit the custom logistic regression model
# The model is fitted using the training data. Gradient descent is used to optimize the model parameters.
gd_log_reg = GradientDescentLogisticReg()
gd_log_reg.fit(pd.DataFrame(X_train), pd.Series(y_train), gamma=0.01, max_iter=1000, eta=0.001)

# Predict the test set
# Predictions are made for the test set using the fitted model.
y_pred = gd_log_reg.prediction(pd.DataFrame(X_test))

# Calculate accuracy
# The accuracy of the model on the test set is calculated.
accuracy = gd_log_reg.accuracy(y_test, y_pred)
print("Accuracy of Gradient Descent Logistic Regression Model:", accuracy)

# Classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Classification report and confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Iteration 100: Loss = 376.97875069407684, Weights = [-1.9960998  -1.35619324  0.20439166 -3.57486356]...
Iteration 200: Loss = 376.42003569032323, Weights = [-2.05407317 -1.23955009  0.00590188 -4.271005  ]...
Iteration 300: Loss = 376.2197876719989, Weights = [-2.08938242 -1.16381854 -0.11648679 -4.6909237 ]...
Stopping early at iteration 368
Accuracy of Gradient Descent Logistic Regression Model: 0.694560669456067
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.81      0.81       197
           1       0.16      0.17      0.16        42

    accuracy                           0.69       239
   macro avg       0.49      0.49      0.49       239
weighted avg       0.70      0.69      0.70       239

Confusion Matrix:
[[159  38]
 [ 35   7]]


### SMOTE + PCA results

In [34]:
# Split the resampled dataset into training and testing sets
# The dataset is split into training and testing sets to evaluate the model.
X_train, X_test, y_train, y_test = train_test_split(X_pca_smote, y_resampled, test_size=0.2, random_state=42)

# Fit the custom logistic regression model
# The model is fitted using the training data. Gradient descent is used to optimize the model parameters.
gd_log_reg = GradientDescentLogisticReg()
gd_log_reg.fit(pd.DataFrame(X_train), pd.Series(y_train), gamma=0.01, max_iter=1000, eta=0.001)

# Predict the test set
# Predictions are made for the test set using the fitted model.
y_pred = gd_log_reg.prediction(pd.DataFrame(X_test))

# Calculate accuracy
# The accuracy of the model on the test set is calculated.
accuracy = gd_log_reg.accuracy(y_test, y_pred)
print("Accuracy of Gradient Descent Logistic Regression Model:", accuracy)

# Classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Classification report and confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Iteration 100: Loss = 8855.44145628098, Weights = [ -1.16266185   0.40878424  -1.16586515 -25.97639856]...
Stopping early at iteration 109
Accuracy of Gradient Descent Logistic Regression Model: 0.5681440443213296
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.57      0.57      1805
           1       0.57      0.57      0.57      1805

    accuracy                           0.57      3610
   macro avg       0.57      0.57      0.57      3610
weighted avg       0.57      0.57      0.57      3610

Confusion Matrix:
[[1026  779]
 [ 780 1025]]


The PCA doesn't improve the results from the SMOTE. Again, the dataset is balanced, but the SMOTE introduced synthetic examples that didn't improve the model as the feature relationships are not well captured. Balanced but poor performance.

### Best model
The downsampling + PCA model is the best-performing model based on these results. It achieves a good balance of overall accuracy (74.9%) and shows an improvement in recall (0.37) for the minority class compared to the other models. The f1-score and precision for class 1 are also higher than the other models, indicating that it makes more successful predictions for delayed shipments while maintaining a reasonable overall accuracy. This suggests that PCA helps retain key information while reducing dimensionality, improving the model's ability to recognize delayed shipments in a more balanced way compared to the purely downsampled or SMOTE models.